# Welcome to the Kitchen Aide Agent 🐝


🎯 Scenario: You need some help figuring out what to make with the food in your fridge and what you might need to buy during your next grocery shop.

Your Kitchen-Aide Agent can assist you by checking to see what foods are in your pantry, regrigerator, and freezer. It also has access to all your personal recipes as well as new ones online.

You can ask Kitchen-Aide questions like:
- Which ingedients am I missing to create X recipe?
- What can I make with my ingredients on hand?
- What food do I have on hand?

## 🔧 Setup
First, let's install the BeeAI Framework and set up our environment.

- setting up the observability so we can capture and log the actions our agent takes

In [1]:
%pip install -Uqq arize-phoenix s3fs unstructured "requests==2.32.4"\
 "openinference-instrumentation-beeai==0.1.13" \
 "beeai-framework[duckduckgo,rag]==0.1.69" "fsspec==2025.3.0" jedi \
 "opentelemetry-api==1.37.0" "opentelemetry-sdk==1.37.0" "grpcio>=1.71.2" "langgraph<=0.5.0"

# The following wraps Notebook output
from IPython.display import HTML, display
def set_css(*_, **__):
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))
get_ipython().events.register("pre_run_cell", set_css)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.4/340.4 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━

Now let's import the necessary modules:


In [2]:
import os
import asyncio
import time
import phoenix as px
import ipywidgets
from typing import Any, Optional
from datetime import date
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.types import RequirementAgentOutput
from beeai_framework.agents.requirement.requirements import Requirement, Rule
from beeai_framework.agents.requirement.requirements.conditional import ConditionalRequirement
from beeai_framework.backend import ChatModel, ChatModelParameters
from beeai_framework.backend.document_loader import DocumentLoader
from beeai_framework.backend.embedding import EmbeddingModel
from beeai_framework.backend.text_splitter import TextSplitter
from beeai_framework.backend.vector_store import VectorStore
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import GlobalTrajectoryMiddleware
from beeai_framework.tools import Tool, ToolRunOptions, tool, StringToolOutput
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
from beeai_framework.tools.search.retrieval import VectorStoreSearchTool
from beeai_framework.tools.think import ThinkTool
from beeai_framework.tools.weather import OpenMeteoTool
from beeai_framework.tools.types import ToolRunOptions
from openinference.instrumentation.beeai import BeeAIInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from langchain_text_splitters import MarkdownHeaderTextSplitter


 ## 1️⃣ LLM Providers: Choose Your AI Engine

BeeAI Framework supports 10+ LLM providers including Ollama, Groq, OpenAI, Watsonx.ai, and more, giving you flexibility to choose local or hosted models based on your needs. You can find the documentation on how to connect to other providers [here](https://framework.beeai.dev/modules/backend).


### *❗* Exercise: Select your Language Model Provider

Change the `provider` and `model` variables to your desired provider and model.

If you select a provider that requires an API key URL or Project_ID, select the key icon on the left menu and set the variables to match those in the userdata.get() function.

Try several models to see how your agent performs. Note that you may need to modify the system prompt for each model, as they all have their own system prompt best practice.

In [3]:
#Use widgets to show provider choices
providers=ipywidgets.ToggleButtons(options=['ollama','openai'])
display(providers)

ToggleButtons(options=('ollama', 'openai'), value='ollama')

In Colab, install and start Ollama for providing the Embedding Model used during RAG operations.

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
provider=providers.value
from google.colab import userdata
# Ollama - No parameters required
if provider=="ollama":
    model="granite4:tiny-h"
    #model="granite3.3"
    provider_model=provider+":"+model
    !ollama pull $model
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=0))
# OpenAI - Place OpenAI API Key in Colab Secrets (key icon) as OPENAI_KEY
elif provider=="openai":
    model="gpt-5-mini"
    provider_model=provider+":"+model
    api_key = userdata.get('OPENAI_KEY')             #Set secret value using key in left menu
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1), api_key=api_key)
else:
  print("Provider " + provider + " undefined")

## Pull Recipe and Food Inventory Data from your Fridge, Freezer, and Pantry into Colab

In [6]:
import os

if os.getcwd() == "/":
    os.chdir('/content/')

if os.path.exists("beeai-workshop"):
    !rm -rf beeai-workshop

# Sparse checkout only kitchen-contents directory
!git clone --filter=blob:none --no-checkout https://github.com/IBM/beeai-workshop.git
os.chdir("beeai-workshop")
!git sparse-checkout init --cone
!git sparse-checkout set kitchen-aide/kitchen-contents
!git checkout

os.chdir("kitchen-aide/kitchen-contents")

print("Now inside:", os.getcwd())


Cloning into 'beeai-workshop'...
remote: Enumerating objects: 1750, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 1750 (delta 197), reused 172 (delta 159), pack-reused 1505 (from 2)
Receiving objects: 100% (1750/1750), 419.62 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (691/691), done.
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 4 (delta 0), pack-reused 3 (from 2)
Receiving objects: 100% (13/13), 64.71 KiB | 974.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.
Your branch is up to date with 'origin/main'.
Now inside: /content/beeai-workshop/kitchen-aide/kitchen-contents


see a sneak peak of each file in your kitchen inentory and your recipes

In [7]:
folder = "/content/beeai-workshop/kitchen-aide/kitchen-contents"
num_lines_to_show = 20

for fname in sorted(os.listdir(folder)):
    fpath = os.path.join(folder, fname)

    if os.path.isfile(fpath):
        print("\n" + "="*80)
        print(f"📄 {fname}  —  showing first {num_lines_to_show} lines")
        print("="*80)

        with open(fpath, "r") as f:
            for i in range(num_lines_to_show):
                line = f.readline()
                if not line:
                    break
                print(line.rstrip())



📄 freezer_contents.md  —  showing first 20 lines
# Freezer Contents

## Food Item
**Item** Blueberries
**Expiration Date** None

## Food Item
**Item** Spinich
**Expiration Date** None

## Food Item
**Item** Ravioli
**Expiration Date** None

## Food Item
**Item** Italian Sauseage
**Expiration Date** None

## Food Item
**Item** Pesto

📄 fridge_contents.md  —  showing first 20 lines
# Fridge Contents

## Food Item
**item** Eggs
**expiration date** None

## Food Item
**item** Milk
**expiration date** None

## Food Item
**item** Bacon
**expiration date** None

## Food Item
**item** Carrots
**expiration date** None

## Food Item
**item** Mozzarella Cheese

📄 my_recipes.md  —  showing first 20 lines
# My Recipes

## Neapolitan Margherita Pizza

**Servings:** 2
**Total Time:** 9 hours
**Tags:** Italian, Pizza", Neapolitan, DInner

### Ingredients

#### Dough
- 300 g Tipo 00 flour (Caputo preferred)
- 195 g water (65% hydration)
- 6 g salt
- 0.2–1 g fresh yeast (or 1/8 tsp active dry yeast)
##

## 2️⃣ Provide System Prompt

### *❗* Exercise: Customize Your System Prompt
Try modifying the system prompt. Customize the "basic rules" section to add your own. Note that changes to the system prompt will affect the performance of the model. Creating a great `System Prompt` is an art, not a science.

In [8]:
instruct_prompt = f"""You help the user understand the inventory of their kitchen and what recipes they can make or what they are lacking.

Basic Rules:
- Favor my recipes over recipes on the internet
- If we cant' prepare a recipe, make a shopping list of the items needed for a recipe
"""

## 3️⃣ Memory Systems: Maintaining Context across iterations or sessions

BeeAI has several built in memory startegies to chose from such as sliding window, token optimized, summarized, and more. Read about them [here](https://framework.beeai.dev/modules/memory.

In [9]:
from beeai_framework.memory import UnconstrainedMemory

memory = UnconstrainedMemory()

## 4️⃣ Tools: Enabling LLMs to Take Action

The BeeAI framework provides [built in tools](https://framework.beeai.dev/modules/tools#built-in-tools) for common tool types, but also provides the ability to create [custom tools](https://framework.beeai.dev/modules/tools#creating-custom-tools).

### Adding Framework Provided Tools
The **Think tool** encourages a Re-Act or planning pattern (depending on how it is implemented in the conditional requirements) where the agent reasons and plans before calling a tool.

The DuckDuckGoSearchTool is a Web Search tool that provides relevant data from the internet to the LLM

In [10]:
think_tool = ThinkTool()
internet_search_tool = DuckDuckGoSearchTool()

### Add A Custom Tool to Read Kitchen Inventory


There are 2 ways to provide custom tools to your agent. For simple tools you can use the `@tool` decorator above the function. For more complex tools, you can extend the `Tool Class` and customize things such as the run time and tool execution.  

We will extend the tool class in this example, because we want to midify the _run method. To learn more about advanced tool customization, take a look at this section in the [documentation](https://framework.beeai.dev/modules/tools#advanced-custom-tool).

In [11]:
class InventoryToolInput(BaseModel):
    pass


class InventoryTool(Tool[InventoryToolInput, ToolRunOptions, StringToolOutput]):
    """
    Base inventory tool. Subclasses define the location and tool name.
    """

    name = "InventoryBase"
    description = "Base inventory tool – do not register directly."
    input_schema = InventoryToolInput

    def __init__(
        self,
        location: str,
        options: dict[str, Any] | None = None,
    ) -> None:
        self.location = location
        super().__init__(options)

    def _create_emitter(self) -> Emitter:
        return Emitter.root().child(
            namespace=["tool", "example", "result"],
            creator=self,
        )

    async def _run(
        self, input: InventoryToolInput, options: ToolRunOptions | None, context: RunContext
    ) -> StringToolOutput:
        location = self.location
        file_path = f"{location}_contents.md"

        try:
            with open(file_path, "r") as file:
                result = file.read()
        except Exception as e:
            result = f"Error reading file {file_path}: {e}"

        print(location, "inventory:", result)
        return StringToolOutput(result=result)


class PantryInventoryTool(InventoryTool):
    name = "get_pantry_inventory"
    description = "Search the pantry inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="pantry", options=options)


class FreezerInventoryTool(InventoryTool):
    name = "get_freezer_inventory"
    description = "Search the freezer inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="freezer", options=options)


class FridgeInventoryTool(InventoryTool):
    name = "get_fridge_inventory"
    description = "Search the fridge inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="fridge", options=options)


#Three distinct tools from the base InventoryTool Class
get_pantry_tool = PantryInventoryTool()
get_freezer_tool = FreezerInventoryTool()
get_fridge_tool = FridgeInventoryTool()


## 5️⃣ Creating a RAG (Retrieval Augmented Generation) Tool to Search My Recipes

The BeeAI Framework has built in abstractions to make RAG simple to implement. Read more about it [here](https://framework.beeai.dev/modules/rag).

First, we must pull an embedding model which converts text into numerical vectors so we can compare meanings and retrieve the most relevant snippets. The original document is:
1. preprocessed (cleaned + broken into chunks)
2. ran through the embedding algorithm
3. stored in the vector database (in system RAM for this example)


In [12]:
!ollama pull nomic-embed-text:latest

In [13]:
embedding_model = EmbeddingModel.from_name("ollama:nomic-embed-text")

Take a look at the internal documents so you know what type of questions to ask your agent


In [14]:
file_path="my_recipes.md"
with open(file_path, 'r') as file:
    content = file.read()

#Print first 50 lines
lines = content.splitlines()
for i in range(min(40, len(lines))):
    print(lines[i])

# My Recipes

## Neapolitan Margherita Pizza

**Servings:** 2
**Total Time:** 9 hours
**Tags:** Italian, Pizza", Neapolitan, DInner

### Ingredients

#### Dough
- 300 g Tipo 00 flour (Caputo preferred)
- 195 g water (65% hydration)
- 6 g salt
- 0.2–1 g fresh yeast (or 1/8 tsp active dry yeast)
#### Sauce
- 200 g canned San Marzano tomatoes (crushed by hand)
- Pinch of salt
#### Toppings
- 150 g fresh mozzarella (fior di latte or bufala), torn into pieces
- Fresh basil leaves
- Extra virgin olive oil

### Instructions
1. **Make dough**: Mix flour and salt. Dissolve yeast in water, then combine. Mix until dough forms.
2. **Knead**: Knead 8–10 minutes until smooth and elastic.
3. **Bulk rise**: Cover and proof 2 hours at room temp.
4. **Ball and ferment**: Divide into 2 dough balls; cold ferment 6–22 hours.
5. **Preheat oven**: Heat pizza oven/stone to 900°F (480°C) or hottest possible.
6. **Shape**: Press dough into rounds — stretch by hand, don’t use a rolling pin.
7. **Assemble**: Spre

Split the documents into chunks based on Markdown formatting and add the documents to the System RAM BeeAI vector store

In [15]:
from beeai_framework.backend.types import Document
from langchain.schema import Document as LangchainDocument
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on=[("#", "Header 1"), ("##", "Header 2")]

# Directly use Langchain's MarkdownHeaderTextSplitter
langchain_text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers = False
    )

# Split the content to get Langchain Document objects
langchain_doc_splits = langchain_text_splitter.split_text(content)


# Convert each Langchain Document to a BeeAI Framework Document
documents = [
    Document(content=split.page_content, metadata=split.metadata)
    for split in langchain_doc_splits
]

print(documents[1])

# Create vector store and add documents
vector_store = VectorStore.from_name(name="beeai:TemporalVectorStore", embedding_model=embedding_model)
await vector_store.add_documents(documents=documents)
print(f"Vector store populated with {len(documents)} documents")


## Alfredo Farfalle with Smoked Salmon  
**Servings:** 6
**Total Time:** 20 minutes
**Tags:** Italian, Dinner, Quick  
### Ingredients
- 12 oz Farfalle Pasta
- 12 oz Alfrado Sauce
- 6 oz Smoked Salmon sliced
- 2 Tbsp Parmesan Cheese  
### Instructions
1. Cook pasta until al dente
2. Place in oven safe dish and miz with alfrado sauce.
3. Break smoked salmon into inch sized pieces and scatter on the top.
4. Prinkle with parmesan cheese and broil in the oven for 3 minutes.
Vector store populated with 6 documents


In [16]:
# Create the vector store search tool
my_recipe_search = VectorStoreSearchTool(vector_store=vector_store)

## Explore Observability: See what is happening under the hood

Create the function that sets up observability using `OpenTelemetry` and [Arize's Phoenix Platform](https://arize.com/docs/phoenix/inferences/how-to-inferences/manage-the-app). There a several ways to view what is happening under the hood of your agent. View the observability documentation [here](https://framework.beeai.dev/modules/observability).

In [17]:
def setup_observability(endpoint: str = "http://localhost:6006/v1/traces") -> None:
    """
    Sets up OpenTelemetry with OTLP HTTP exporter and instruments the beeai framework.
    """
    resource = Resource(attributes={})
    tracer_provider = trace_sdk.TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
    trace_api.set_tracer_provider(tracer_provider)

    BeeAIInstrumentor().instrument()

In [18]:
load_dotenv()
# Enable OpenTelemetry integration
setup_observability("http://localhost:6006/v1/traces")
px_session = px.launch_app()

/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit https://8f3jjyrgfou7-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


## 6️⃣ Conditional Requirements: Guiding Agent Behavior


What Are Conditional Requirements?
[Conditional requirements](https://framework.beeai.dev/experimental/requirement-agent#conditional-requirement) ensure your agents are reliable by controlling when and how tools are used. They're like business rules for agent behavior. You can make them as strict (esentially writing a static workflow) or flexible (no rules! LLM decides) as you'd like.

The rules that you enforce may seem simple in the BeeAI framework, but in other frameworks they require ~5X the amount of code. Check out this [blog](https://beeai.dev/blog/reliable-ai-agents) where we built the same agent in BeeAI and other agent framework LangGraph.

These conditional requirements enforce the following in only 6 lines of code:
1. The agent must call the think tool as the first tool call and then after every other tool enforcing a re-act pattern. It is not allowed to call the think tool consecutive times in a row.
3. The internet_search_tool can also only be called after the my_recipe_search tool and think tool, it is allowed to be called up to 3 times, and it has a relative priority of 15. Consecutive calls are allowed.
4. The get_fridge_tool tool can only be called once.
5. The get_freezer_tool can be called only after the get_fridge_tool and only can be invoked once.
6. The get_pantry_tool can only be called after the get_fridge_tool and only invoked once.
7. The my_recipe_search is allowed to be invoked up to 6 tiems and allwed to be called consecutively.



##  7️⃣ Assemble Your Reliable BeeAI Agent

This is the part we've been working towards! Let's assemble the agent with all the parts we created.

NEED TO ADD BACK IN MY_RECIPE SEARCH ONCE RAG IS FIXED. AND NEED TO ADD BACK IN THE ONLY AFTER CONDITION FOR DUCK DUCK GO

In [19]:
agent = RequirementAgent(
    llm=llm,
    instructions= instruct_prompt,
    memory = memory,
    tools=[think_tool, internet_search_tool, get_pantry_tool, get_freezer_tool, get_fridge_tool, my_recipe_search],
    requirements=[
        ConditionalRequirement(think_tool, consecutive_allowed=False, force_at_step=1 ),
        ConditionalRequirement(get_pantry_tool, only_after=[get_fridge_tool,think_tool], min_invocations=1, max_invocations=1, priority=10),
        ConditionalRequirement(get_freezer_tool, only_after=[get_fridge_tool,think_tool], min_invocations=1, max_invocations=1, priority=5 ),
        ConditionalRequirement(get_fridge_tool, min_invocations=1, priority=15, max_invocations=1 ),
        ConditionalRequirement(my_recipe_search, only_after=think_tool, max_invocations=6, consecutive_allowed=True),
        ConditionalRequirement(internet_search_tool, max_invocations=3, only_after= [think_tool, my_recipe_search], consecutive_allowed=True),
    ],
    # Log intermediate steps to the console
    middlewares=[GlobalTrajectoryMiddleware(included=[Tool])],
)

### *❗* Exercise: Test Your Agent
Change the execution settings and see what happens. Does your agent run out of iterations? Every task is different and its important to balance flexibility with control.

Example Questions:
- Which of my recipes can I make with the ingredients I have on hand?
- What do I need to buy to make blueberry scones?
- What breakfast recipes can I make with the items I have on hand?
- What internet recipe can I prepare with the items I have on hand?
- What items do I need to buy to prepare my pizza recipe?

In [ ]:
while (question := input("How can Kitchen-Aide Help you? (Enter q to end): ")) !="q":
    response = await agent.run(question, max_retries_per_step=3, total_max_retries=25)
    print(response.last_message.text)

How can Kitchen-Aide Help you? (Enter q to end): Which of my recipes can I make with the ingredients I have on hand?
--> 🛠️ ThinkTool[think][start]: {"input": {"input": {"thoughts": "To determine which recipes you can make, we need to know what ingredients are currently in your pantry, freezer, and fridge. Could you please provide a list of all the items you have on hand?"}}}
<-- 🛠️ ThinkTool[think][success]: "OK"
--> 🛠️ FridgeInventoryTool[get_fridge_inventory][start]: {"input": {"input": {}}}
fridge inventory: # Fridge Contents

## Food Item
**item** Eggs
**expiration date** None

## Food Item
**item** Milk
**expiration date** None

## Food Item
**item** Bacon
**expiration date** None

## Food Item
**item** Carrots
**expiration date** None

## Food Item
**item** Mozzarella Cheese
**expiration date** None

## Food Item
**item** Cream Cheese
**expiration date** None

# Food Item
**item** Ground Beef
**expiration date** None

# Food Item
**item** Onions
**expiration date** None

## Food

In [ ]:
px_session.view()